In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/hse-aml/natural-language-processing.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 670, done.
remote: Total 670 (delta 0), reused 0 (delta 0), pack-reused 670
Receiving objects: 100% (670/670), 233.69 KiB | 441.00 KiB/s, done.
Resolving deltas: 100% (384/384), done.
/content/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo
AWS-tutorial.md  Docker-tutorial.md  README.md		    week2
common		 honor		     setup_google_colab.py  week3
docker		 project	     week1		    week4


In [ ]:
import sys
sys.path.append("..")
from common.download_utils import download_week1_resources

download_week1_resources()

In [ ]:
from week1.grader import Grader

In [ ]:
grader = Grader()

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from ast import literal_eval
import pandas as pd
import numpy as np

In [ ]:
def read_data(filename):
    data = pd.read_csv(filename, sep='\t')
    data['tags'] = data['tags'].apply(literal_eval)
    return data

In [ ]:
train = read_data('data/train.tsv')
validation = read_data('data/validation.tsv')
test = pd.read_csv('data/test.tsv', sep='\t')

In [ ]:
train.head()

,title,tags
0,How to draw a stacked dotplot in R?,[r]
1,mysql select all records where a datetime fiel...,"[php, mysql]"
2,How to terminate windows phone 8.1 app,[c#]
3,get current time in a specific country via jquery,"[javascript, jquery]"
4,Configuring Tomcat to Use SSL,[java]


In [ ]:
X_train, y_train = train['title'].values, train['tags'].values
X_val, y_val = validation['title'].values, validation['tags'].values
X_test = test['title'].values

In [ ]:
import re

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    text = text.lower()
    text = re.sub(REPLACE_BY_SPACE_RE,'',text)
    text = re.sub(BAD_SYMBOLS_RE,'',text)
    text = " ".join([word for word in text.split(" ") if word not in (STOPWORDS)])
    text = re.sub(' +', ' ', text)

    return text

In [ ]:
print(text_prepare("SQL Server - any equivalent of Excel's CHOOSE function?"))

sql server equivalent excels choose function


In [ ]:
def test_text_prepare():
    examples = ["SQL Server - any equivalent of Excel's CHOOSE function?",
                "How to free c++ memory vector<int> * arr?"]
    answers = ["sql server equivalent excels choose function", 
               "free c++ memory vectorint arr"]
    for ex, ans in zip(examples, answers):
        if text_prepare(ex) != ans:
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [ ]:
print(test_text_prepare())

Basic tests are passed.


In [ ]:
prepared_questions = []
for line in open('data/text_prepare_tests.tsv', encoding='utf-8'):
    line = text_prepare(line.strip())
    prepared_questions.append(line)
text_prepare_results = '\n'.join(prepared_questions)

grader.submit_tag('TextPrepare', text_prepare_results)

Current answer for task TextPrepare is:
 sqlitephp readonly
creating multiple textboxes dynamically
self one prefer javascript
save php date ...


In [ ]:

X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [ ]:
X_train[:3]

['draw stacked dotplot r',
 'mysql select records datetime field less specified value',
 'terminate windows phone 81 app']

In [ ]:
tags_counts = {}
words_counts = {}

from collections import Counter
tags_counts = Counter([tag for taglist in y_train for tag in taglist])
words_counts = Counter([word for question in X_train for word in question.split(' ')])

In [ ]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]

grader.submit_tag('WordsTagsCount', '%s\n%s' % (','.join(tag for tag, _ in most_common_tags), 
                                                ','.join(word for word, _ in most_common_words)))

Current answer for task WordsTagsCount is:
 javascript,c#,java
using,php,java...


In [ ]:
DICT_SIZE = 5000
####### YOUR CODE HERE #######
WORDS_TO_INDEX = {word[0]:int_idx for int_idx, word in enumerate(sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:DICT_SIZE])}
####### YOUR CODE HERE #######
INDEX_TO_WORDS = dict(zip(WORDS_TO_INDEX.values(),WORDS_TO_INDEX.keys()))
ALL_WORDS = WORDS_TO_INDEX.keys()

def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'

    """
    result_vector = np.zeros(dict_size)

    words = text.split(" ")
    for word in words:
      if word in words_to_index.keys():
        result_vector[words_to_index[word]]+=1
    
    return result_vector

In [ ]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    for ex, ans in zip(examples, answers):
        # print(ans, " ",my_bag_of_words(ex, words_to_index, 4))
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [ ]:
print(test_my_bag_of_words())

Basic tests are passed.


In [ ]:

from scipy import sparse as sp_sparse

In [ ]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (100000, 5000)
X_val shape  (30000, 5000)
X_test shape  (20000, 5000)


In [ ]:
row = X_train_mybag[10].toarray()[0]
non_zero_elements_count = np.sum([1 for item in row if item != 0])

grader.submit_tag('BagOfWords', str(non_zero_elements_count))

Current answer for task BagOfWords is:
 7...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    
    
    tfidf_vectorizer = TfidfVectorizer(analyzer="word",token_pattern = '(\S+)',min_df=5,max_df=0.9,ngram_range=(1,2))
    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)
    
    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

In [ ]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

In [ ]:
all([tag in tfidf_vocab for tag in ['c++','c#']])

True

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [ ]:
def train_classifier(X_train, y_train):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    # Create and fit LogisticRegression wraped into OneVsRestClassifier.

    ######################################
    ######### YOUR CODE HERE #############
    ######################################


    log_reg = OneVsRestClassifier(LogisticRegression(C=1,penalty='l2',dual=False,max_iter=1000,random_state=42))
    log_reg.fit(X_train,y_train)

    return log_reg



In [ ]:
classifier_mybag = train_classifier(X_train_mybag, y_train)
classifier_tfidf = train_classifier(X_train_tfidf, y_train)

In [ ]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

In [ ]:
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_tfidf)
y_val_inversed = mlb.inverse_transform(y_val)
for i in range(3):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_val[i],
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

Title:	odbc_exec always fail
True labels:	php,sql
Predicted labels:	


Title:	access base classes variable within child class
True labels:	javascript
Predicted labels:	


Title:	contenttype applicationjson required rails
True labels:	ruby,ruby-on-rails
Predicted labels:	ruby-on-rails




In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

In [ ]:
def print_evaluation_scores(y_val, predicted):
    print('Acuracy score: {:.2f}'.format(accuracy_score(y_val, predicted)))
    # F1-Score
    print('F1-Score macro-average: {:.2f}'.format(f1_score(y_val,predicted,average='macro')))
    print('F1-Score micro-average: {:.2f}'.format(f1_score(y_val,predicted,average='micro')))
    print('F1-Score weighted-average: {:.2f}'.format(f1_score(y_val,predicted,average='weighted')))
    # Precision
    print('Precision macro-average: {:.2f}'.format(average_precision_score(y_val,predicted,average='macro')))
    print('Precision micro-average: {:.2f}'.format(average_precision_score(y_val,predicted,average='micro')))
    print('Precision weighted-average: {:.2f}'.format(average_precision_score(y_val,predicted,average='weighted')))




In [ ]:
print('Bag-of-words')
print_evaluation_scores(y_val, y_val_predicted_labels_mybag)
print('Tfidf')
print_evaluation_scores(y_val, y_val_predicted_labels_tfidf)

Bag-of-words
Acuracy score: 0.35
F1-Score macro-average: 0.50
F1-Score micro-average: 0.67
F1-Score weighted-average: 0.64
Precision macro-average: 0.34
Precision micro-average: 0.48
Precision weighted-average: 0.50
Tfidf
Acuracy score: 0.33
F1-Score macro-average: 0.44
F1-Score micro-average: 0.64
F1-Score weighted-average: 0.61
Precision macro-average: 0.30
Precision micro-average: 0.45
Precision weighted-average: 0.48


In [ ]:
def grid_search(c,penalty,X_train_mybag,X_train_tfidf,X_val_mybag,X_val_tfidf,y_train,y_val):
    
    # Define model
    clf = OneVsRestClassifier(LogisticRegression(C = c, penalty = penalty))
    
    # Fit and predict mybag
    clf_mybag = clf.fit(X_train_mybag,y_train)
    y_val_predicted_labels_mybag = clf_mybag.predict(X_val_mybag)
 
    # Fit and predict tfidf
    clf_tfidf = clf.fit(X_train_tfidf,y_train)
    y_val_predicted_labels_tfidf = clf_tfidf.predict(X_val_tfidf)
    
    # Evaluate
    print("For parameters C={}, penalty={}".format(c,penalty))
    print("mybag: F1-score = {:.2f}".format(f1_score(y_val, y_val_predicted_labels_mybag, average = 'weighted'
)))
    print("tfidf: F1-score = {:.2f}".format(f1_score(y_val, y_val_predicted_labels_tfidf, average = 'weighted'
)))


    
from itertools import product
C = [0.1,1,10,100]
regularization = ['l2']
grid = list(product(C,regularization))

for c,reg in grid:
    grid_search(c,reg,X_train_mybag,X_train_tfidf,X_val_mybag,X_val_tfidf,y_train,y_val)

For parameters C=0.1, penalty=l2
mybag: F1-score = 0.58
tfidf: F1-score = 0.38


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


KeyboardInterrupt: ignored

In [ ]:
clf = OneVsRestClassifier(LogisticRegression(C = 1, penalty = 'l2',dual=False,random_state=42,tol=0.000,max_iter=1000))
clf_mybag = clf.fit(X_train_mybag,y_train)
test_predictions = clf.predict(X_test_mybag)
test_pred_inversed = mlb.inverse_transform(test_predictions)

test_predictions_for_submission = '\n'.join('%i\t%s' % (i, ','.join(row)) for i, row in enumerate(test_pred_inversed))
grader.submit_tag('MultilabelClassification', test_predictions_for_submission)

Current answer for task MultilabelClassification is:
 0	mysql,php
1	javascript
2	
3	javascript,jquery
4	android,java
5	php,xml
6	ajax,json,web-services
7	...


In [ ]:
def print_words_for_tag(classifier, tag, tags_classes, index_to_words, all_words):
    """
        classifier: trained classifier
        tag: particular tag
        tags_classes: a list of classes names from MultiLabelBinarizer
        index_to_words: index_to_words transformation
        all_words: all words in the dictionary
        
        return nothing, just print top 5 positive and top 5 negative words for current tag
    """
    print('Tag:\t{}'.format(tag))
    
    # Extract an estimator from the classifier for the given tag.
    # Extract feature coefficients from the estimator. 
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    coef = classifier.coef_[tags_classes.index(tag)]
    coef_dict = {idx:c for idx,c in enumerate(coef.tolist())}
    sorted_coefs = sorted(coef_dict.items(), key=lambda x: x[1], reverse=True)
    
    # top-5 words sorted by the coefficiens.
    top_positive_words = [index_to_words[i[0]] for i in sorted_coefs[:5]]
    # bottom-5 words  sorted by the coefficients.
    top_negative_words = [index_to_words[i[0]] for i in sorted_coefs[-5:]]
    print('Top positive words:\t{}'.format(', '.join(top_positive_words)))
    print('Top negative words:\t{}\n'.format(', '.join(top_negative_words)))

In [ ]:
print_words_for_tag(classifier_tfidf, 'c', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)
print_words_for_tag(classifier_tfidf, 'c++', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)
print_words_for_tag(classifier_tfidf, 'linux', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)

Tag:	c
Top positive words:	c, cc++, scanf, malloc, pointer
Top negative words:	c#, javascript, python, php, java

Tag:	c++
Top positive words:	c++, qt, cc++, boost, mfc
Top negative words:	c#, python, javascript, php, java

Tag:	linux
Top positive words:	linux, ubuntu, c, address, signal
Top negative words:	method, array, jquery, c#, javascript



In [ ]:
 grader.status()

You want to submit these parts:
Task TextPrepare:
 sqlitephp readonly
creating multiple textboxes dynamically
self one prefer javascript
save php date ...
Task WordsTagsCount:
 javascript,c#,java
using,php,java...
Task BagOfWords:
 7...
Task MultilabelClassification:
 0	mysql,php
1	javascript
2	
3	javascript,jquery
4	android,java
5	php,xml
6	ajax,json,web-services
7	...


In [ ]:
STUDENT_EMAIL = "sumanth.cs18@bmsce.ac.in"
STUDENT_TOKEN = "2bqySreum19Sklxr"
grader.status()

You want to submit these parts:
Task TextPrepare:
 sqlitephp readonly
creating multiple textboxes dynamically
self one prefer javascript
save php date ...
Task WordsTagsCount:
 javascript,c#,java
using,php,java...
Task BagOfWords:
 7...
Task MultilabelClassification:
 0	mysql,php
1	javascript
2	
3	javascript,jquery
4	android,java
5	php,xml
6	ajax,json,web-services
7	...


In [ ]:

grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
